スクリプト

In [ ]:
#スクリプト

#python make_mut_document.py CosmicNCV.tsv hg38.fa

import pandas as pd
from Bio import SeqIO
import re
import sys

mut_file=sys.argv[1]
ref_fasta=sys.argv[2]

df_cosmic=pd.read_table(mut_file)
df_cut=df_cosmic[['ID_SAMPLE', 'ID_tumour','genome position','WT_SEQ', 'MUT_SEQ']]


#レファレンスファイルについて
list_id=[]
list_desc=[]
list_seq=[]

#biopythonを用いて、idやdesc、seqを抽出
for record in SeqIO.parse(ref_fasta, 'fasta'):
    id_part = record.id
    desc_part = record.description
    seq = record.seq

    list_id.append(id_part)
    list_desc.append(desc_part)
    list_seq.append(seq)

#mutについて、genome positiionから、chrとsingle positionに分ける
list_cut_chr=[]
list_cut_pos=[]
for i in range(len(df_cut)):
    list_cut_chr.append((re.split('[:-]',df_cut['genome position'][i]))[0])
    list_cut_pos.append((re.split('[:-]',df_cut['genome position'][i]))[1])
df_cut['chr']=pd.Series(list_cut_chr)
df_cut['single position']=pd.Series(list_cut_pos)

#mutのdfからchr23、24、MUT_SEQのnanを除く

df_cut_chr=df_cut[(df_cut['chr'] != '23') & (df_cut['chr'] != '24') & (df_cut['chr'] != '25')]
df_cut_chr.dropna(subset=['MUT_SEQ'],inplace=True)

#indexを振り直す
df_cut_chr_i=df_cut_chr.reset_index()

#使わないdfを削除
del df_cut_chr,df_cut

#refから変異位置の前後の配列を抽出
seq_before=[]
seq_after=[]
for i in range(len(df_cut_chr_i)):
    seq_before.append(list_seq[list_id.index('chr'+df_cut_chr_i['chr'][i])][int(df_cut_chr_i['single position'][i])-2])
    seq_after.append(list_seq[list_id.index('chr'+df_cut_chr_i['chr'][i])][int(df_cut_chr_i['single position'][i])])

df_cut_chr_i['seq_before']=pd.Series(seq_before)
df_cut_chr_i['seq_after']=pd.Series(seq_after)
    
#大文字に変更
df_cut_chr_i['seq_before']=df_cut_chr_i['seq_before'].str.upper()
df_cut_chr_i['seq_after']=df_cut_chr_i['seq_after'].str.upper()

#3文字の配列にする（seq_forwardが鋳型）
df_cut_chr_i['seq_forward']=df_cut_chr_i['seq_before']+df_cut_chr_i['MUT_SEQ']+df_cut_chr_i['seq_after']
df_cut_chr_i['seq_forward_r']=df_cut_chr_i['seq_after']+df_cut_chr_i['MUT_SEQ']+df_cut_chr_i['seq_before']

#辞書型で相補鎖に変換
dict_base={"A":"T","G":"C","C":"G","T":"A"}

list_seq_reverse=[]
list_wt_seq_reverse=[]
for i in range(len(df_cut_chr_i)):
    list_seq_reverse.append((df_cut_chr_i['seq_forward_r'][i]).translate(str.maketrans(dict_base)))
    list_wt_seq_reverse.append((str(df_cut_chr_i['WT_SEQ'][i])).translate(str.maketrans(dict_base)))
df_cut_chr_i['seq_reverse']=pd.Series(list_seq_reverse)
df_cut_chr_i['WT_seq_reverse']=pd.Series(list_wt_seq_reverse)

#ID_tumorのカウント
dict_tumor_id=df_cut_chr_i['ID_tumour'].value_counts().to_dict()

#以下、ID_tumorごとに取り出し変異をテキストに書き込む
df_index_for_id=pd.DataFrame()
for i in list_tumor_id:
    df_index_for_id['index_{}'.format(i)]=pd.Series((df_cut_chr_i[df_cut_chr_i['ID_tumour']==i]).index)
    for j in df_index_for_id['index_{}'.format(i)]:
        if pd.isnull(j)==True:
            continue
        else:
            with open("text/{}.txt".format(i), "a") as f:
                if (df_cut_chr_i.iloc[int(j)])['WT_SEQ']=='C'or (df_cut_chr_i.iloc[int(j)])['WT_SEQ']=='G':
                    f.write((df_cut_chr_i.iloc[int(j)])['WT_SEQ']+'>'+(df_cut_chr_i.iloc[int(j)])['MUT_SEQ']+\
                    '_'+(df_cut_chr_i.iloc[int(j)])['seq_forward'][0:1]+\
                    (df_cut_chr_i.iloc[int(j)])['seq_forward'][2:3]+' ')
                else:
                    f.write((df_cut_chr_i.iloc[int(j)])['WT_seq_reverse']+'>'+(df_cut_chr_i.iloc[int(j)])['seq_reverse'][1:2]+\
                    '_'+(df_cut_chr_i.iloc[int(j)])['seq_reverse'][0:1]+\
                    (df_cut_chr_i.iloc[int(j)])['seq_reverse'][2:3]+' ')


In [1]:
import pandas as pd
from Bio import SeqIO
import re
import sys

In [2]:
mut_file='CosmicNCV.tsv'
ref_fasta='hg38.fa'

df_cosmic=pd.read_table(mut_file)
df_cut=df_cosmic[['ID_SAMPLE', 'ID_tumour','genome position','WT_SEQ', 'MUT_SEQ']]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


In [3]:
df_cut.head()

,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ
0,2634366,2494762,3:191870081-191870081,A,G
1,2634366,2494762,4:175610288-175610288,G,T
2,2634602,2494998,10:113312456-113312456,G,T
3,2634732,2495128,7:49183066-49183066,A,T
4,2634732,2495128,20:13631885-13631885,G,A


In [4]:
#リファレンスファイルについて
list_id=[]
list_desc=[]
list_seq=[]

#biopythonを用いて、idやdesc、seqを抽出
for record in SeqIO.parse(ref_fasta, 'fasta'):
    id_part = record.id
    desc_part = record.description
    seq = record.seq

    list_id.append(id_part)
    list_desc.append(desc_part)
    list_seq.append(seq)


In [5]:
len(list_id)

455

In [6]:
list_id

['chr1',
 'chr10',
 'chr11',
 'chr11_KI270721v1_random',
 'chr12',
 'chr13',
 'chr14',
 'chr14_GL000009v2_random',
 'chr14_GL000225v1_random',
 'chr14_KI270722v1_random',
 'chr14_GL000194v1_random',
 'chr14_KI270723v1_random',
 'chr14_KI270724v1_random',
 'chr14_KI270725v1_random',
 'chr14_KI270726v1_random',
 'chr15',
 'chr15_KI270727v1_random',
 'chr16',
 'chr16_KI270728v1_random',
 'chr17',
 'chr17_GL000205v2_random',
 'chr17_KI270729v1_random',
 'chr17_KI270730v1_random',
 'chr18',
 'chr19',
 'chr1_KI270706v1_random',
 'chr1_KI270707v1_random',
 'chr1_KI270708v1_random',
 'chr1_KI270709v1_random',
 'chr1_KI270710v1_random',
 'chr1_KI270711v1_random',
 'chr1_KI270712v1_random',
 'chr1_KI270713v1_random',
 'chr1_KI270714v1_random',
 'chr2',
 'chr20',
 'chr21',
 'chr22',
 'chr22_KI270731v1_random',
 'chr22_KI270732v1_random',
 'chr22_KI270733v1_random',
 'chr22_KI270734v1_random',
 'chr22_KI270735v1_random',
 'chr22_KI270736v1_random',
 'chr22_KI270737v1_random',
 'chr22_KI270738v1_ra

In [7]:
#mutについて、genome positiionから、chrとsingle positionに分ける
list_cut_chr=[]
list_cut_pos=[]
for i in range(len(df_cut)):
    list_cut_chr.append((re.split('[:-]',df_cut['genome position'][i]))[0])
    list_cut_pos.append((re.split('[:-]',df_cut['genome position'][i]))[1])
df_cut['chr']=pd.Series(list_cut_chr)
df_cut['single position']=pd.Series(list_cut_pos)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
df_cut.head()

,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position
0,2634366,2494762,3:191870081-191870081,A,G,3,191870081
1,2634366,2494762,4:175610288-175610288,G,T,4,175610288
2,2634602,2494998,10:113312456-113312456,G,T,10,113312456
3,2634732,2495128,7:49183066-49183066,A,T,7,49183066
4,2634732,2495128,20:13631885-13631885,G,A,20,13631885


In [9]:
#mutのdfからchr23、24、MUT_SEQのnanを除く

df_cut_chr=df_cut[(df_cut['chr'] != '23') & (df_cut['chr'] != '24') & (df_cut['chr'] != '25')]
df_cut_chr.dropna(subset=['MUT_SEQ'],inplace=True)
df_cut_chr.dropna(subset=['WT_SEQ'],inplace=True)

#indexを振り直す
df_cut_chr_i=df_cut_chr.reset_index()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [10]:
df_cut_chr_i

,index,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position
0,0,2634366,2494762,3:191870081-191870081,A,G,3,191870081
1,1,2634366,2494762,4:175610288-175610288,G,T,4,175610288
2,2,2634602,2494998,10:113312456-113312456,G,T,10,113312456
3,3,2634732,2495128,7:49183066-49183066,A,T,7,49183066
4,4,2634732,2495128,20:13631885-13631885,G,A,20,13631885
5,5,2633825,2494221,10:55869458-55869458,C,T,10,55869458
6,6,2634325,2494721,7:107774762-107774762,G,T,7,107774762
7,7,2340684,2205667,16:80055352-80055352,A,G,16,80055352
8,8,2688338,2547320,5:145211773-145211773,T,A,5,145211773
9,9,2340644,2205627,21:14009329-14009329,T,C,21,14009329


In [11]:
#refから変異位置の前後の配列を抽出
seq_before=[]
seq_after=[]
for i in range(len(df_cut_chr_i)):
    seq_before.append(list_seq[list_id.index('chr'+df_cut_chr_i['chr'][i])][int(df_cut_chr_i['single position'][i])-2])
    seq_after.append(list_seq[list_id.index('chr'+df_cut_chr_i['chr'][i])][int(df_cut_chr_i['single position'][i])])

In [12]:
#df_cut_chr_iにつなげる
df_cut_chr_i['seq_before']=pd.Series(seq_before)
df_cut_chr_i['seq_after']=pd.Series(seq_after)

In [13]:
df_cut_chr_i.head()

,index,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position,seq_before,seq_after
0,0,2634366,2494762,3:191870081-191870081,A,G,3,191870081,A,G
1,1,2634366,2494762,4:175610288-175610288,G,T,4,175610288,c,g
2,2,2634602,2494998,10:113312456-113312456,G,T,10,113312456,A,A
3,3,2634732,2495128,7:49183066-49183066,A,T,7,49183066,A,C
4,4,2634732,2495128,20:13631885-13631885,G,A,20,13631885,t,a


In [14]:
#大文字に変更
df_cut_chr_i['seq_before']=df_cut_chr_i['seq_before'].str.upper()
df_cut_chr_i['seq_after']=df_cut_chr_i['seq_after'].str.upper()

In [15]:
df_cut_chr_i.head()

,index,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position,seq_before,seq_after
0,0,2634366,2494762,3:191870081-191870081,A,G,3,191870081,A,G
1,1,2634366,2494762,4:175610288-175610288,G,T,4,175610288,C,G
2,2,2634602,2494998,10:113312456-113312456,G,T,10,113312456,A,A
3,3,2634732,2495128,7:49183066-49183066,A,T,7,49183066,A,C
4,4,2634732,2495128,20:13631885-13631885,G,A,20,13631885,T,A


In [16]:
#3文字の配列にする（seq_forwardが鋳型）
df_cut_chr_i['seq_forward']=df_cut_chr_i['seq_before']+df_cut_chr_i['MUT_SEQ']+df_cut_chr_i['seq_after']
df_cut_chr_i['seq_forward_r']=df_cut_chr_i['seq_after']+df_cut_chr_i['MUT_SEQ']+df_cut_chr_i['seq_before']

In [17]:
df_cut_chr_i.head()

,index,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position,seq_before,seq_after,seq_forward,seq_forward_r
0,0,2634366,2494762,3:191870081-191870081,A,G,3,191870081,A,G,AGG,GGA
1,1,2634366,2494762,4:175610288-175610288,G,T,4,175610288,C,G,CTG,GTC
2,2,2634602,2494998,10:113312456-113312456,G,T,10,113312456,A,A,ATA,ATA
3,3,2634732,2495128,7:49183066-49183066,A,T,7,49183066,A,C,ATC,CTA
4,4,2634732,2495128,20:13631885-13631885,G,A,20,13631885,T,A,TAA,AAT


In [18]:
#辞書型で相補鎖に変換
dict_base={"A":"T","G":"C","C":"G","T":"A"}

list_seq_reverse=[]
list_wt_seq_reverse=[]
for i in range(len(df_cut_chr_i)):
    list_seq_reverse.append((df_cut_chr_i['seq_forward_r'][i]).translate(str.maketrans(dict_base)))
    list_wt_seq_reverse.append((str(df_cut_chr_i['WT_SEQ'][i])).translate(str.maketrans(dict_base)))
df_cut_chr_i['seq_reverse']=pd.Series(list_seq_reverse)
df_cut_chr_i['WT_seq_reverse']=pd.Series(list_wt_seq_reverse)

In [19]:
df_cut_chr_i.head()

,index,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position,seq_before,seq_after,seq_forward,seq_forward_r,seq_reverse,WT_seq_reverse
0,0,2634366,2494762,3:191870081-191870081,A,G,3,191870081,A,G,AGG,GGA,CCT,T
1,1,2634366,2494762,4:175610288-175610288,G,T,4,175610288,C,G,CTG,GTC,CAG,C
2,2,2634602,2494998,10:113312456-113312456,G,T,10,113312456,A,A,ATA,ATA,TAT,C
3,3,2634732,2495128,7:49183066-49183066,A,T,7,49183066,A,C,ATC,CTA,GAT,T
4,4,2634732,2495128,20:13631885-13631885,G,A,20,13631885,T,A,TAA,AAT,TTA,C


In [31]:
'AC'.translate(str.maketrans(dict_base))

'TG'

In [20]:
#ID_tumorのカウント
dict_tumor_id=df_cut_chr_i['ID_tumour'].value_counts().to_dict()
list_tumor_id = list(dict_tumor_id.keys())

In [26]:
#以下、ID_tumorごとに取り出し変異をテキストに書き込む
df_index_for_id=pd.DataFrame()
for i in list_tumor_id:
    df_index_for_id['index_{}'.format(i)]=pd.Series((df_cut_chr_i[df_cut_chr_i['ID_tumour']==i]).index)
    for j in df_index_for_id['index_{}'.format(i)]:
        if pd.isnull(j)==True:
            continue
        else:
            with open("text/{}.txt".format(i), "a") as f:
                if (df_cut_chr_i.iloc[int(j)])['WT_SEQ']=='C'or (df_cut_chr_i.iloc[int(j)])['WT_SEQ']=='G':
                    f.write((df_cut_chr_i.iloc[int(j)])['WT_SEQ']+'>'+(df_cut_chr_i.iloc[int(j)])['MUT_SEQ']+\
                    '_'+(df_cut_chr_i.iloc[int(j)])['seq_forward'][0:1]+\
                    (df_cut_chr_i.iloc[int(j)])['seq_forward'][2:3]+' ')
                else:
                    f.write((df_cut_chr_i.iloc[int(j)])['WT_seq_reverse']+'>'+(df_cut_chr_i.iloc[int(j)])['seq_reverse'][1:2]+\
                    '_'+(df_cut_chr_i.iloc[int(j)])['seq_reverse'][0:1]+\
                    (df_cut_chr_i.iloc[int(j)])['seq_reverse'][2:3]+' ')

KeyboardInterrupt: 

if,elseの確認、テスト

In [27]:
if (df_cut_chr_i.iloc[int(0)])['WT_SEQ']=='C'or (df_cut_chr_i.iloc[int(0)])['WT_SEQ']=='G':
    print(1)
else:
    print(2)

2


In [28]:
if (df_cut_chr_i.iloc[int(1)])['WT_SEQ']=='C'or (df_cut_chr_i.iloc[int(1)])['WT_SEQ']=='G':
    print(1)
else:
    print(2)

1


In [29]:
(df_cut_chr_i.iloc[int(1)])['WT_SEQ']

'G'

In [21]:
(df_cut_chr_i.iloc[int(0)])['WT_SEQ']

'A'

In [22]:
(df_cut_chr_i.iloc[int(0)])['WT_SEQ']=='C'

False

In [23]:
(df_cut_chr_i.iloc[int(0)])['WT_SEQ']=='G'

False

In [25]:
(df_cut_chr_i.iloc[int(0)])['WT_SEQ']=='A'

True

In [30]:
(df_cut_chr_i.iloc[int(0)])['WT_SEQ']=='C'or (df_cut_chr_i.iloc[int(0)])['WT_SEQ']=='G'

False

In [ ]:
#以下、ID_tumorごとに取り出し変異をテキストに書き込む
df_index_for_id=pd.DataFrame()
for i in list_tumor_id:
    df_index_for_id['index_{}'.format(i)]=pd.Series((df_cut_chr_i[df_cut_chr_i['ID_tumour']==i]).index)
    for j in df_index_for_id['index_{}'.format(i)]:
        if pd.isnull(j)==True:
            continue
        else:
            with open("text/{}.txt".format(i), "a") as f:
                if (df_cut_chr_i.iloc[int(j)])['WT_SEQ']=='C'or (df_cut_chr_i.iloc[int(j)])['WT_SEQ']=='T':
                    f.write((df_cut_chr_i.iloc[int(j)])['WT_SEQ']+'>'+(df_cut_chr_i.iloc[int(j)])['MUT_SEQ']+\
                    '_'+(df_cut_chr_i.iloc[int(j)])['seq_forward'][0:1]+\
                    (df_cut_chr_i.iloc[int(j)])['seq_forward'][2:3]+' ')
                elif(df_cut_chr_i.iloc[int(j)])['WT_SEQ']=='A'or (df_cut_chr_i.iloc[int(j)])['WT_SEQ']=='G':
                    f.write((df_cut_chr_i.iloc[int(j)])['WT_seq_reverse']+'>'+(df_cut_chr_i.iloc[int(j)])['seq_reverse'][1:2]+\
                    '_'+(df_cut_chr_i.iloc[int(j)])['seq_reverse'][0:1]+\
                    (df_cut_chr_i.iloc[int(j)])['seq_reverse'][2:3]+' ')
                else:
                    continue

In [35]:
df_cut_chr_i.head()

,index,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position,seq_before,seq_after,seq_forward,seq_forward_r,seq_reverse,WT_seq_reverse
0,0,2634366,2494762,3:191870081-191870081,A,G,3,191870081,A,G,AGG,GGA,CCT,T
1,1,2634366,2494762,4:175610288-175610288,G,T,4,175610288,C,G,CTG,GTC,CAG,C
2,2,2634602,2494998,10:113312456-113312456,G,T,10,113312456,A,A,ATA,ATA,TAT,C
3,3,2634732,2495128,7:49183066-49183066,A,T,7,49183066,A,C,ATC,CTA,GAT,T
4,4,2634732,2495128,20:13631885-13631885,G,A,20,13631885,T,A,TAA,AAT,TTA,C


In [37]:
df_cut_chr_i['ID_tumour'][0]

2494762

In [ ]:
2580164

In [40]:
df_cut_chr_i[df_cut_chr_i['ID_tumour']==2580164]

,index,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position,seq_before,seq_after,seq_forward,seq_forward_r,seq_reverse,WT_seq_reverse
16305605,18386088,2721343,2580164,5:1295113-1295114,GG,AA,5,1295113,A,G,AAAG,GAAA,CTTT,CC


In [38]:
df_test_616173=df_cut_chr_i[df_cut_chr_i['ID_tumour']==616173]

In [39]:
df_test_616173

,index,ID_SAMPLE,ID_tumour,genome position,WT_SEQ,MUT_SEQ,chr,single position,seq_before,seq_after,seq_forward,seq_forward_r,seq_reverse,WT_seq_reverse
505,570,688013,616173,10:69993003-69993003,G,A,10,69993003,C,G,CAG,GAC,CTG,C
4485,5052,688013,616173,12:90491427-90491427,G,A,12,90491427,T,A,TAA,AAT,TTA,C
5061,5705,688013,616173,8:137337801-137337801,A,T,8,137337801,C,C,CTC,CTC,GAG,T
6771,7627,688013,616173,21:40995609-40995609,T,A,21,40995609,C,T,CAT,TAC,ATG,A
6772,7628,688013,616173,17:48012301-48012301,C,A,17,48012301,G,C,GAC,CAG,GTC,G
7346,8275,688013,616173,11:72177575-72177575,C,A,11,72177575,A,C,AAC,CAA,GTT,G
7897,8899,688013,616173,12:24694481-24694481,A,T,12,24694481,T,T,TTT,TTT,AAA,T
8506,9584,688013,616173,15:81646172-81646172,G,T,15,81646172,T,C,TTC,CTT,GAA,C
9102,10245,688013,616173,2:41137871-41137871,A,G,2,41137871,T,G,TGG,GGT,CCA,T
9722,10945,688013,616173,3:193093284-193093284,C,A,3,193093284,C,T,CAT,TAC,ATG,G
